# 🎭 Device Mismatch

In this notebook, we'll cover mismatch, the physical reality that IC designs will be manufactured with imperfections, how to model and design around these variations in analog design.

🎥 Video Insight:

<iframe width="560" height="315" src="https://www.youtube.com/embed/TFBzCRLSaG0?si=dZdkP9NKbZYypITP" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

##### 📝 Notes:

The basic relationship in modelling mismatch is given by:

$$ \sigma^2(\Delta P) = \dfrac{A_P^2}{WL} + S_P^2 D_x^2 $$

Or, the variance of the difference of a given parameter between two identical devices is inversely proportional to the device's area (determined by constant $A_P$) and is proportional to the physical distance between the two devices on a given wafer.

This makes for two basic observations:

- The bigger the device, the less variation it has
- The closer devices are, the more similar they're going to be

##### 🧠 Exercises

1. Repeat the derivation in the videos, indicate clearly how you arrive at the equation, could you repeat a similar derivation for another set-up?
2. Follow the mismatch simulation below, try tweaking its parameters, what do you notice?

In [ ]:
# Import the hdl21 library
import hdl21 as h
from hdl21.prefix import u
import vlsirtools.spice as vsp

# Import the SKY130 PDK
from sitepdks import *
import gf180_hdl21 as g
from gf180_hdl21 import primitives as p

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Don't touch any of this!
def get_sim(device : h.Module,vgs=1.8) -> h.sim.Sim:

    @h.sim.sim
    class DeviceSim:

        @h.module
        class Tb:

            # Set up our ports
            VSS = h.Port()
            VDD = h.Signal()
            
            # Define our voltages
            vdd = h.DcVoltageSource(dc=vgs)(p=VDD, n=VSS)
            sweep = h.PulseVoltageSource(delay=0,
                                         v1=0,
                                         v2=5,
                                         period=1,
                                         rise=1,
                                         fall=1,
                                         width=1)(n=VSS)
            # Instantiate our device
            DUT = device(d=sweep.p, g=VDD, s=VSS, b=VSS)

        tran = h.sim.Tran(tstop=1,tstep=0.001)
        inc1 = g.install.include_design()
        inc2 = g.install.include_mos(h.pdk.CmosCorner.TT)
        par1 = h.sim.Param(name="sw_stat_global", value=1)
        par2 = h.sim.Param(name="sw_stat_mismatch", value=1)

    return DeviceSim

opts = vsp.SimOptions(
            simulator=vsp.SupportedSimulators.NGSPICE,
            fmt=vsp.ResultFormat.SIM_DATA,
            rundir="./scratch",
        )

import numpy as np
import matplotlib.pyplot as plt

# Modify vgs, W/L and simulate!
vgs=1.8
w, l = 0.6 * u, 0.6 * u
sim = get_sim(device=p.NFET_3p3V(w=w, l=l),vgs=vgs)
rvs = sim.run(opts=opts)

results = rvs[vsp.sim_data.AnalysisType.TRAN]

# Plot the results
x = results.data['v(xtop.sweep_p)']
y = -results.data['i(v.xtop.vsweep)']*1000

plt.xlabel('Vds (V)')
plt.ylabel('Ids (mA)')
plt.title(f'I-V Curve for {w.number}/{l.number} 3.3V NMOS at Vgs={vgs}V')

plt.plot(x,y)